In [8]:
import torch
import sklearn as sk
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.io as sio

### Load raw data

In [78]:
arbor_dict = sio.loadmat("../data/raw/M_arbor_data_50k_4Apr23.mat")
arbors = arbor_dict["hist_ax_de_api_bas"]
arbor_ids = pd.DataFrame(data = {"specimen_id": arbor_dict["specimen_id"], "arbor_index": range(len(arbor_dict["specimen_id"]))})
anno_csv = pd.read_csv("../data/raw/exc_inh_ME_fMOST_EM_specimen_ids_shuffled_4Apr23.txt")
arbor_ids["specimen_id"] = arbor_ids["specimen_id"].str.strip()
anno_csv["specimen_id"] = anno_csv["specimen_id"].str.strip()
anno = pd.merge(anno_csv, arbor_ids, on = "specimen_id", how = "left").drop("Unnamed: 0", axis = 1).query("M_cell")
anno

,specimen_id,platform,class,group,class_id,T_cell,E_cell,M_cell,subgroup,proofread,arbor_index
0,864691135274125285_199010,EM,exc,1,0.0,False,False,True,1,NaN,0
2,864691135212658304_427884,EM,exc,1,0.0,False,False,True,1,NaN,2
3,864691135354962383_392595,EM,exc,1,0.0,False,False,True,1,NaN,3
5,864691136656750077_610633,EM,exc,1,0.0,False,False,True,1,NaN,5
6,864691136898656366_396172,EM,exc,1,0.0,False,False,True,1,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...
52252,864691135750158472_388862,EM,exc,1,0.0,False,False,True,1,NaN,52252
52254,864691135373371848_297143,EM,exc,1,0.0,False,False,True,1,NaN,52254
52255,864691135714363796_557060,EM,exc,1,0.0,False,False,True,1,NaN,52255
52256,864691135587516402_491694,EM,exc,1,0.0,False,False,True,1,NaN,52256


In [79]:
pd.pivot_table(anno, index = "platform", columns = "class", values = "arbor_index", aggfunc = "count")

class,exc,inh
platform,,
EM,43314.0,141.0
ME,244.0,203.0
fMOST,280.0,NaN
patchseq,831.0,788.0


### Classify raw data